# Desafio 1

O desafio 1 é sobre a extração, limpeza e manipulação de dados no INEP sobre o Censo da Educação Superior. 

#### DADOS 

Estão disponíveis no site do Inep os Microdados do Censo da Educação Superior 2018 (DM_IES, DM_CURSO, DM_DOCENTE, DM_ALUNO, DM_LOCAL_OFERTA e TB_AUX_AREA_OCDE) em formato CSV delimitados por Pipe ( | ). Os arquivos encontram-se compactados (em formato .zip) pelo software 7-zip.  


#### ANEXO I – Dicionários de dados e Tabelas Auxiliares 

Contém, em formato .xlsx (Excel), o Dicionário de Dados do Censo da Educação Superior 2018 e também uma tabela auxiliar com o código e o nome dos países: 

1.DICIONÁRIO DE DADOS 

- TABELA DE ALUNO 
- TABELA DE CURSO 
- TABELA DE IES 
- TABELA DE LOCAL DE OFERTA 
- TABELA DE DOCENTE 
- TABELA AUXILIAR OCDE 
 
2.TABELA CONTENDO O NOME DO PAÍS DE ORIGEM OU NATURALIZAÇÃO 
 
#### ANEXO II – Questionários do Censo da Educação Superior  

Contém, em formato .pdf (Portable Document Format), os seguintes questionários do Censo da Educação Superior 2018 e estão disponíveis para download na pasta anexos: 

- MÓDULO IES 
- MÓDULO CURSO 
- MÓDULO DOCENTE 
- MÓDULO ALUNO 

### Leitura e preparação inicial dos dados

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import random
import glob
import scipy.stats as stats

KeyboardInterrupt: 

In [ ]:
#Lendo total de linhas de DM_ALUNO sem importar o dataset
with open ('dados/DM_ALUNO.csv') as f:
    total_linhas = sum(1 for row in f)-1
    
total_linhas

In [ ]:
#Gerando um array de números aleatórios de 1 até total_linhas de tamanho = drop (99%)
percent_amostra = 0.01
amostra = int(percent_amostra * total_linhas)
drop = total_linhas - amostra

skip = random.sample(range(1,total_linhas),drop)

In [ ]:
#Lendo o dataset com skiprows = 99% dos dados, gerando uma amostra de 1%. 
df_alunos = pd.read_csv('dados/DM_ALUNO.csv', sep='|', encoding='latin1', skiprows=skip)
df_alunos.shape

In [ ]:
#Lendo os demais datasets
df_ies = pd.read_csv('dados/DM_IES.CSV', sep='|', encoding='latin1')
df_curso = pd.read_csv('dados/DM_CURSO.CSV', sep='|', encoding='latin1')
df_docente = pd.read_csv('dados/DM_DOCENTE.CSV', sep='|', encoding='latin1')
df_local = pd.read_csv('dados/DM_LOCAL_OFERTA.CSV', sep='|', encoding='latin1')
df_cine = pd.read_csv('dados/TB_AUX_CINE_BRASIL.CSV', sep='|', encoding='latin1')

##### 1) Crie duas tabelas: uma com número de universidades públicas e privadas por estado e outra com número de alunos por universidade 

Universidades Públicas e Privadas por estado:

In [ ]:
#Usando a tabela do IBGE para atrelar código do estado a UF.
estados = pd.read_excel("dados/Tabela Estados IBGE.xlsx")
estados.head()

In [ ]:
estados = estados.rename(columns={'Código da UF':'CO_UF','UF':'SG_UF'}).drop('Estado',axis=1)
estados.head()

In [ ]:
#Função que define universidade pública ou privada a partir de TP_CATEGORIA_ADMINISTRATIVA.
def publi_privada(categoria_adm):
    if categoria_adm < 3:
        return 'Universidade Pública'
    elif categoria_adm != 7:
        return 'Universidade Privada'

In [ ]:
#Agrega estados ao df de uni / Aplica a função criando nova coluna como resposta / Agrupa por estado e tipo somando IES
df_publi_privada = df_ies.merge(estados)
df_publi_privada['TIPO_UNI'] = df_publi_privada['TP_CATEGORIA_ADMINISTRATIVA'].apply(publi_privada)
df_publi_privada.groupby(['SG_UF','TIPO_UNI']).agg(TOTAL_UNI = ('CO_IES','nunique'))

Número de alunos por universidade:

In [ ]:
#Agega o nome da universidade / Agrupa pelo nome da uni e conta o número de alunos ordenando de forma descendente
df_alunos_uni = df_alunos.merge(df_ies[['CO_IES','NO_IES']])
df_alunos_uni = df_alunos_uni.groupby('NO_IES', as_index=False).agg(
                                                N_ALUNOS = ('ID_ALUNO','nunique')).sort_values(by='N_ALUNOS',ascending=False)
df_alunos_uni.head()

In [ ]:
#Como é uma amostra de 1% do total, pode-se multiplicar por 1/percent_amostra para se aproximar do número total de alunos
df_alunos_uni['N_ALUNOS'] = (df_alunos_uni['N_ALUNOS'] * 1/percent_amostra).astype(int)
df_alunos_uni

##### 2) Pergunta-se: é verdade que existem menos mulheres nos cursos de exatas? Explique com os dados.

In [ ]:
display(df_cine[['CO_CINE_AREA_GERAL','NO_CINE_AREA_GERAL']].drop_duplicates())

In [ ]:
#Nota-se que 5, 6 e 7 são cursos de exatas.
exatas = [5,6,7]

In [ ]:
#Agrega o nome e código da área ao df_alunos e atribui a df_exatas
df_exatas = df_alunos.merge(df_cine[['CO_CINE_ROTULO', 'CO_CINE_AREA_GERAL', 'NO_CINE_AREA_GERAL']])

In [ ]:
#Mantém em df_exatas somente os alunos que o cód da área esteja em exatas.
df_exatas = df_exatas.loc[df_exatas['CO_CINE_AREA_GERAL'].isin(exatas),['ID_ALUNO','TP_SEXO','NO_CINE_AREA_GERAL']]
df_exatas.head()

In [ ]:
#Para garantir que df_exatas não tem alunos duplicados. 
df_exatas = df_exatas.drop_duplicates(subset='ID_ALUNO')
df_exatas.head()

In [ ]:
print(f'Total de mulheres nos cursos de exatas',df_exatas.loc[df_exatas['TP_SEXO']==1,'ID_ALUNO'].count())
print(f'Total de homens nos cursos de exatas',df_exatas.loc[df_exatas['TP_SEXO']==2,'ID_ALUNO'].count())

De fato existem menos mulheres do que homens nos cursos de exatas.

##### 3) Quantos cursos novos abrem por ano? 

In [ ]:
df_novos_cursos = df_curso.dropna(subset=['DT_INICIO_FUNCIONAMENTO']).copy()
df_novos_cursos['DT_INICIO_FUNCIONAMENTO'].isnull().sum()

In [ ]:
#df_novos_cursos['DT_INICIO_FUNCIONAMENTO'] = pd.to_datetime(df_novos_cursos['DT_INICIO_FUNCIONAMENTO'],format='%d/%m/%Y')
#Ao tentar converter a coluna para data, nota-se que há um erro de digitação em uma das linhas. 
df_novos_cursos['DT_INICIO_FUNCIONAMENTO'] = df_novos_cursos['DT_INICIO_FUNCIONAMENTO'].str.replace('2917','2017')

In [ ]:
#Agora sim, convertendo a coluna e criando uma nova apenas com o ano de inicio.
df_novos_cursos['DT_INICIO_FUNCIONAMENTO'] = pd.to_datetime(df_novos_cursos['DT_INICIO_FUNCIONAMENTO'],format='%d/%m/%Y')
df_novos_cursos['ANO_INICIO_FUNCIONAMENTO'] = df_novos_cursos['DT_INICIO_FUNCIONAMENTO'].dt.year

In [ ]:
#Agrupando pelo ano e inicio e contando os cursos
df_novos_cursos = (df_novos_cursos.groupby('ANO_INICIO_FUNCIONAMENTO', as_index=False).
                       agg(N_NOVOS_CURSOS = ('CO_CURSO','count')).sort_values(by='ANO_INICIO_FUNCIONAMENTO', ascending=False))

In [ ]:
#Considerando um periodo de 10 anos.
anos = 10
ult = df_novos_cursos['ANO_INICIO_FUNCIONAMENTO'].max()
df_novos_cursos = df_novos_cursos.loc[df_novos_cursos['ANO_INICIO_FUNCIONAMENTO'] >= ult - anos]
df_novos_cursos

In [ ]:
print(f'Abrem em média {int(df_novos_cursos["N_NOVOS_CURSOS"].mean())} novos cursos por ano')

##### 4) A afirmação a seguir é verdadeira: Alunos da região Norte têm maior tendência a não concluírem os cursos quando comparados ao restante do país? Comprove com dados.

Considerando desistente os alunos que trancaram a matrícula ou estão desvinculados do curso (códigos 3 e 4).

In [ ]:
#Agrega código de UF e IES ao df_alunos em df_desistente / Desistente = 1 - Não desistente = 0 /Remove alunos duplicados
df_desistente = df_alunos.merge(df_ies[['CO_IES','CO_UF']])
df_desistente['DESISTENTE'] = np.where((df_desistente['TP_SITUACAO'] == 3) | (df_desistente['TP_SITUACAO'] == 4),1,0)
df_desistente = df_desistente.drop_duplicates(subset=['ID_ALUNO'])

Estados da Região Norte e respecitvos CO_UF:
- Acre: 12
- Amapá: 16
- Amazonas: 13
- Pará: 15
- Rondônia: 11
- Roraima: 14
- Tocantins: 17

In [ ]:
#Separando em dois dataframes alunos da região norte e demais regiões.
norte = [11,12,13,14,15,16,17]
df_alunos_norte = df_desistente.loc[df_desistente['CO_UF'].isin(norte)]
df_alunos_outras = df_desistente.loc[~df_desistente['CO_UF'].isin(norte)]
df_alunos_norte.shape, df_alunos_outras.shape

In [ ]:
print('O indice de desistência da região Norte é de {:.2f} %'.
      format((df_alunos_norte.loc[df_alunos_norte['DESISTENTE']==1, 'DESISTENTE'].count() * 100) / df_alunos_norte.shape[0]))
print('O indice de desistência de outras regiões é de {:.2f} %'.
      format((df_alunos_outras.loc[df_alunos_outras['DESISTENTE']==1, 'DESISTENTE'].count() * 100) / df_alunos_outras.shape[0]))

A afirmação é falsa.

###### 5) Crie uma variável que represente a taxa de abandono para cada IES. É correto afirmar professores mais/menos capacitados influenciam tal taxa?

In [ ]:
#Removendo professores duplicados / Agrupando por IES / Tirando a média de escolaridade dos professores
df_prof = df_docente.drop_duplicates(subset='ID_DOCENTE')
df_prof = df_prof.groupby('CO_IES', as_index = False).agg(ESCOLARIDADE = ('TP_ESCOLARIDADE','mean'))

In [ ]:
#Agrupando alunos desistentes por IES / Calculando Taxa de abondono 
df_desistente = df_desistente.groupby('CO_IES', as_index=False).agg(TAXA_ABANDONO = ('DESISTENTE','mean'))

In [ ]:
# Juntando os os dois dataframes
df_aluno_docente = df_prof.merge(df_desistente)

In [ ]:
#Calculando a correlação entre a escolaridade do professor e a taxa de abandono. 
pearson, p_pearson = stats.pearsonr(df_aluno_docente['ESCOLARIDADE'], df_aluno_docente['TAXA_ABANDONO'])
spearman, p_spearman = stats.spearmanr(df_aluno_docente['ESCOLARIDADE'], df_aluno_docente['TAXA_ABANDONO'])

print(f'Correlação de Pearson: {round(pearson, 2)}, com p-valor de {round(p_pearson, 3)}')
print(f'Correlação de Spearman: {round(spearman, 2)}, com p-valor de {round(p_spearman, 3)}')

- Como o p-valor da correlação de Pearson e de Spearman é menor do que 0.05 há um fraca correlação.
- A correlação negativa nos diz que quanto maior a escolaridade, menor a taxa de abandono nas IES.

##### 6) Quais os cursos com maior crescimento de matriculas por região? E quais os com maior queda? Como você explicaria isso.

In [ ]:
# Criando mapa de estados por região
regiao = {'Norte' : [11, 12, 13, 14, 15, 16, 17],
          'Nordeste' : [21, 22, 23 ,24, 25, 26, 27, 28, 29],
          'Centro-Oeste' : [50, 51, 52, 53],
          'Sudeste' : [31, 32, 33, 35],
          'Sul' : [41, 42, 43]}

In [ ]:
#Trazendo as colunas com código e nome do curso e código do estado
df_matriculas = df_alunos.merge(df_curso[['CO_CURSO','NO_CURSO','CO_UF']])

In [ ]:
#Função para classificar a região do curso a partir do CO_UF
def reg_estado(co_uf, regiao):
    for reg in regiao.keys():
        if co_uf in regiao[reg]:
            return reg
        
df_matriculas['REGIAO'] = df_matriculas['CO_UF'].apply(reg_estado, args=(regiao,))

In [ ]:
#Agrupando por região, curso e ano. / Calculando o número de mátriculas por região, curso e ano. 
df_matriculas = df_matriculas.groupby(['REGIAO','NO_CURSO','NU_ANO_INGRESSO'], as_index=False).agg\
                                                                                        (N_MATRICULAS = ('ID_ALUNO','nunique'))
df_matriculas['N_MATRICULAS'] = df_matriculas['N_MATRICULAS'] / percent_amostra

In [ ]:
#Filtrando os anos de 2017 e 2018
df_matriculas = df_matriculas.loc[df_matriculas['NU_ANO_INGRESSO'].isin([2018,2017])]

In [ ]:
#Criando colunas separadas com matrículas de 2017 e 2018 / Renomeando colunas / Dropando valores nulos
df_matriculas = df_matriculas.pivot(index=['REGIAO','NO_CURSO'], columns=['NU_ANO_INGRESSO'],values=['N_MATRICULAS'])
df_matriculas = df_matriculas.reset_index()
df_matriculas.columns=['REGIAO','NO_CURSO','MATRICULAS_2017','MATRICULAS_2018']
df_matriculas.dropna(inplace=True)

In [ ]:
#Calculando a difereça de uma ano pra outro
df_matriculas['AUMENTO_MATRICULAS'] = df_matriculas['MATRICULAS_2018'] - df_matriculas['MATRICULAS_2017']
df_matriculas

In [ ]:
#Criando dataframes para maiores e menores matriculas por região. 
df_maiores = pd.DataFrame(columns=df_matriculas.columns)
df_menores = pd.DataFrame(columns=df_matriculas.columns)

In [ ]:
#Append nos 5 maiores cursos de cada região
for regiao in df_matriculas['REGIAO'].unique():
    aux = df_matriculas.loc[df_matriculas['REGIAO'] == regiao]
    df_maiores = df_maiores.append(aux.nlargest(5, 'AUMENTO_MATRICULAS'))
    df_menores = df_menores.append(aux.nsmallest(5, 'AUMENTO_MATRICULAS'))

In [ ]:
df_maiores

In [ ]:
df_menores